In [ ]:
from abc import abstractmethod
from math import pi

In [ ]:
class Square:
    def __init__(self, side):
        self.side = side

    def area(self):
        pass

    def perimeter(self):
        pass

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):
        pass

    def perimeter(self):
        pass

In [ ]:
class Visitor:

    @abstractmethod
    def visit_square(self, component):
        pass

    @abstractmethod
    def visit_circle(self, component):
        pass

In [ ]:
class AreaVisitor(Visitor):

    def visit_square(self, component):
        return component.side ** 2

    def visit_circle(self, component):
        return pi * component.radius ** 2


class PerimeterVisitor(Visitor):

    def visit_square(self, component):
        return component.side * 4

    def visit_circle(self, component):
        return 2 * pi * component.radius

In [ ]:
class GeometricShape:

    @abstractmethod
    def accept(self, visitor: Visitor):
        pass

In [13]:
class Square(GeometricShape):
    def __init__(self, side):
        self.side = side

    def accept(self, visitor: Visitor):
        return visitor.visit_square(self)


class Circle(GeometricShape):
    def __init__(self, radius):
        self.radius = radius

    def accept(self, visitor: Visitor):
        return visitor.visit_circle(self)


In [16]:
shape_components = [Square(4), Circle(3)]

area_visitor = AreaVisitor()
perimeter_visitor = PerimeterVisitor()

for shape in shape_components:
    print(f"Area: {shape.accept(area_visitor)}")
    print(f"Perimeter: {shape.accept(perimeter_visitor)}")

Area: 16
Perimeter: 16
Area: 28.274333882308138
Perimeter: 18.84955592153876
